In [1]:
import numpy as np
from sentence_transformers import SentenceTransformer
from text_untils import *
import pickle
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
with open('output/bm25/bm25plus.pkl', 'rb') as f:
    bm25plus = pickle.load(f)

In [3]:
def load_docs_from_file(file_path):
    docs = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                doc = json.loads(line)
                docs.append(doc)
            except json.JSONDecodeError:
                continue

    return docs

docs = load_docs_from_file('output/bm25/docs.jsonl')

In [4]:
query = "Đi xe máy không đội mũ bảo hiểm bị phạt bao nhiêu tiền?"
query = clean_text(query)
query = word_segment(query)
query = remove_stopword(normalize_text(query))

query_bm25 = query.split()
print(query_bm25)

['đi', 'xe_máy', 'đội', 'mũ', 'bảo_hiểm', 'phạt', 'bao_nhiêu', 'tiền']


In [5]:
top_docs = bm25plus.get_top_n(query_bm25, docs, n=100)
bm25_scores = bm25plus.get_scores(query)
doc_scores = np.sort(bm25_scores)[::-1][:100]

In [6]:
for doc_dict, score in zip(top_docs, doc_scores):
    doc_id = doc_dict['id']
    doc_title = doc_dict['title']
    print(f"ID: {doc_id}, Document: {doc_title}, Score: {score}")

ID: 6663569cc2f544363eec469b, Document: đi xe_máy đội mũ bảo_hiểm lái_xe phạt bao_nhiêu tiền, Score: 349.73307725695133
ID: 6663573bc2f544363eeea8e6, Document: phạt đội mũ bảo_hiểm đi xe_máy, Score: 348.910813820627
ID: 6663569cc2f544363eec510a, Document: xe_máy đội mũ bảo_hiểm phạt bao_nhiêu, Score: 348.32608544151094
ID: 6663569ac2f544363eec2487, Document: trường_hợp đội mũ bảo_hiểm đi xe_máy phạt, Score: 347.38645647042216
ID: 666356a0c2f544363eec8ace, Document: đi xe_máy đội mũ bảo_hiểm phạt thế_nào, Score: 347.0957296729322
ID: 6663569fc2f544363eec876d, Document: trường_hợp đội mũ bảo_hiểm đi xe_máy phạt, Score: 347.0957296729322
ID: 6663569cc2f544363eec4985, Document: phạt lỗi đi xe_máy đội mũ bảo_hiểm 2024 bao_nhiêu, Score: 347.0957296729322
ID: 6663569ac2f544363eec2486, Document: đội mũ bảo_hiểm phạt bao_nhiêu tiền, Score: 346.54357155517465
ID: 6663569fc2f544363eec8555, Document: đội mũ bảo_hiểm giấy_tờ xe_máy phạt bao_nhiêu, Score: 344.762855512981
ID: 6663569dc2f544363eec641

In [7]:
# query_sbert = query.replace('_', ' ') # for keepitreal/vietnamese-sbert
query_sbert = query # for bkai/vietnamese-bi-encoder
print(query_sbert)

đi xe_máy đội mũ bảo_hiểm phạt bao_nhiêu tiền


In [8]:
from pymongo import MongoClient
from bson import ObjectId
import pandas as pd

def get_selected_rows_from_mongodb(database_name, collection_name, top_docs):
    client = MongoClient('mongodb://localhost:27017/')
    db = client[database_name]
    collection = db[collection_name]

    doc_ids = [ObjectId(doc_dict['id']) for doc_dict in top_docs]

    cursor = collection.find({'_id': {'$in': doc_ids}})
    data = list(cursor)
    df = pd.DataFrame(data)

    # client.close()
    return df

In [9]:
selected_rows = get_selected_rows_from_mongodb('lawlaboratory', 'questions_cleaned', top_docs)

In [10]:
selected_rows.sample(5)

,_id,title,description,date_answer,field,source_url,conclusion,reference,quote
99,6663573bc2f544363eeeaab0,Đi xe máy không đội mũ bảo hiểm có buộc lập bi...,Chào tổ tư vấn. Vừa rồi chú tôi điều khiển xe ...,2019-11-27,Vi phạm hành chính,https://thuvienphapluat.vn/hoi-dap-phap-luat/4...,[Căn cứ Điểm d Khoản 4 Điều 8 Nghị định 46/201...,None,None
25,6663569bc2f544363eec2f98,Lỗi không đội mũ bảo hiểm có bị giữ bằng không?,None,2024-06-03,Giao thông vận tải,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,"[Theo đó, tại Nghị định 100/2019/NĐ-CP không q...",Căn cứ Điều 125 Luật Xử lý vi phạm hành chính ...,"{'name': 'Điều 82. Tạm giữ phương tiện, giấy t..."
38,6663569cc2f544363eec3f77,"Có bị xử phạt khi đi xe máy chở ba người, khôn...",Hôm trước nhà mình có người bà bị bệnh phải ch...,2021-12-06,Giao thông vận tải,https://thuvienphapluat.vn/hoi-dap-phap-luat/5...,[Căn cứ Khoản 1 Điều 30 Luật giao thông đường ...,None,None
97,6663573ac2f544363eeea0b6,Có được nộp phạt tại chỗ khi chạy xe máy không...,Ban biên tập cho tôi hỏi: Trong trường hợp chạ...,2020-03-10,Vi phạm hành chính,https://thuvienphapluat.vn/hoi-dap-phap-luat/4...,"[Tại Điểm i, Điểm k Khoản 2 Điều 6 Nghị định 1...",None,None
33,6663569bc2f544363eec36f1,"17 tuổi chạy xe máy vượt tốc độ, không đội mũ ...","17 tuổi chạy xe máy vượt tốc độ 10km/h, không ...",2021-03-03,Giao thông vận tải,https://thuvienphapluat.vn/hoi-dap-phap-luat/5...,"[Cơ sở xử phạt: Nghị định 100/2019/NĐ-CP., Lỗi...",None,None


In [11]:
sentences_title = []
ids_title = []

for idx in selected_rows.index:
    sentences_title.append(selected_rows.at[idx, 'title'])
    ids_title.append(selected_rows.at[idx, '_id'])

In [12]:
segmented_sentences_title = [word_segment(sentence) for sentence in sentences_title]

In [13]:
import torch
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [14]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained('bkai-foundation-models/vietnamese-bi-encoder')
model = AutoModel.from_pretrained('bkai-foundation-models/vietnamese-bi-encoder')
model = model.to('cuda')

In [15]:
encoded_input_title = tokenizer(sentences_title, padding=True, truncation=True, return_tensors='pt')
encoded_input_title = {key: value.to('cuda') for key, value in encoded_input_title.items()}

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [16]:
with torch.no_grad():
    model_output = model(**encoded_input_title)

In [17]:
embeddings_title = mean_pooling(model_output, encoded_input_title['attention_mask'])
embeddings_title = embeddings_title.cpu().numpy()

In [18]:
encoded_query = tokenizer([query_sbert], padding=True, truncation=True, return_tensors='pt')
encoded_query = {key: value.to('cuda') for key, value in encoded_query.items()}

In [19]:
with torch.no_grad():
    model_output_query = model(**encoded_query)

embeddings_query = mean_pooling(model_output_query, encoded_query['attention_mask'])
embeddings_query = embeddings_query.cpu().numpy()

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

similarities_title = cosine_similarity(embeddings_query, embeddings_title)

In [21]:
top_20_indices_title = np.argsort(similarities_title[0])[::-1][:20]

for idx in top_20_indices_title:
    similarity_score = similarities_title[0][idx]
    similar_sentence = sentences_title[idx]
    doc_id = ids_title[idx]
    print(f"ID: {doc_id}, Similarity Score: {similarity_score:.4f}, Sentence: {similar_sentence}")

ID: 6663569dc2f544363eec6414, Similarity Score: 0.7394, Sentence: Điều khiển xe máy không đội mũ bảo hiểm bị phạt bao nhiêu?
ID: 6663569cc2f544363eec510a, Similarity Score: 0.7346, Sentence: Người ngồi trên xe máy không đội mũ bảo hiểm bị phạt bao nhiêu?
ID: 6663569fc2f544363eec856b, Similarity Score: 0.7206, Sentence: Xe đạp điện không đội mũ bảo hiểm phạt bao nhiêu tiền?
ID: 666356a0c2f544363eec8ace, Similarity Score: 0.7181, Sentence: Đi xe máy không đội mũ bảo hiểm bị phạt thế nào?
ID: 6663569cc2f544363eec4985, Similarity Score: 0.7014, Sentence: Mức phạt lỗi đi xe máy không đội mũ bảo hiểm 2024 là bao nhiêu?
ID: 6663569dc2f544363eec53e2, Similarity Score: 0.6971, Sentence: Có phạt đối với người ngồi trên xe máy không đội mũ bảo hiểm không?
ID: 6663569fc2f544363eec7b41, Similarity Score: 0.6903, Sentence: Mức phạt khi người ngồi trên xe máy điện không đội mũ bảo hiểm
ID: 6663569ac2f544363eec23c3, Similarity Score: 0.6798, Sentence: Người đi xe đạp điện không đội mũ bảo hiểm bị phạt

In [22]:
top_20_indices_title_ids = []
for idx in top_20_indices_title:
    top_20_indices_title_ids.append(ids_title[idx])

In [23]:
selected_rows_title = selected_rows[selected_rows['_id'].isin(top_20_indices_title_ids)]
selected_rows_title.sample(5)

,_id,title,description,date_answer,field,source_url,conclusion,reference,quote
88,666356a0c2f544363eec8acf,Đi xe đạp không đội mũ bảo hiểm phạt bao nhiêu...,None,2022-04-25,Giao thông vận tải,https://thuvienphapluat.vn/hoi-dap-phap-luat/5...,[Bên cạnh đó theo Khoản 4 Điều 8 Nghị định 100...,Bên cạnh đó theo Khoản 4 Điều 8 Nghị định 100/...,None
59,6663569ec2f544363eec67d7,Ngồi trên xe máy không đội mũ bảo hiểm bị phạt...,Ngồi trên xe máy không đội mũ bảo hiểm bị phạt...,2016-08-30,Giao thông vận tải,https://thuvienphapluat.vn/hoi-dap-phap-luat/1...,[Theo quy định tại điểm i khoản 3 Điều 6 Nghị ...,None,None
31,6663569bc2f544363eec3553,Đi xe máy không đội mũ bảo hiểm bị xử phạt bao...,Đi xe máy không đội mũ bảo hiểm bị xử phạt bao...,2016-08-30,Giao thông vận tải,https://thuvienphapluat.vn/hoi-dap-phap-luat/1...,[Theo quy định tại điểm i khoản 3 Điều 6 Nghị ...,None,None
97,6663573ac2f544363eeea0b6,Có được nộp phạt tại chỗ khi chạy xe máy không...,Ban biên tập cho tôi hỏi: Trong trường hợp chạ...,2020-03-10,Vi phạm hành chính,https://thuvienphapluat.vn/hoi-dap-phap-luat/4...,"[Tại Điểm i, Điểm k Khoản 2 Điều 6 Nghị định 1...",None,None
35,6663569bc2f544363eec3bf5,Trẻ 15 tuổi điều khiển xe máy không đội mũ bảo...,"Xin chào, tôi có vấn đề cần giải đáp như sau: ...",2019-02-26,Giao thông vận tải,https://thuvienphapluat.vn/hoi-dap-phap-luat/4...,"[Thứ nhất, về hành vi không đội mũ bảo hiểm kh...",None,None


In [24]:
def merge_quote(quote):
    if isinstance(quote, dict):
        name = quote.get('name', ' ')
        content = ' '.join(quote.get('content', []))

        if name is None:
            name = ' '
        if content is None:
            content = ' '

        return f"{name}. {content}"
    else:
        return None

selected_rows_title['conclusion'] = selected_rows_title.apply(lambda row: merge_quote(row['quote']) if row['conclusion'] == [] else row['conclusion'], axis=1)

C:\Users\Admin\AppData\Local\Temp\ipykernel_21016\224380773.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows_title['conclusion'] = selected_rows_title.apply(lambda row: merge_quote(row['quote']) if row['conclusion'] == [] else row['conclusion'], axis=1)


In [25]:
def create_answer(row):
    conclusion_value = row['conclusion']

    if isinstance(conclusion_value, list):
        conclusion_value = ' '.join(conclusion_value)
    elif pd.isna(conclusion_value):
        conclusion_value = ''

    if pd.isna(row['quote']):
        return conclusion_value
    else:
        reference = row['reference'] if not pd.isna(row['reference']) else ''
        merged_quote = merge_quote(row['quote'])
        return f"{reference} {merged_quote}"

selected_rows_title['answer'] = selected_rows_title.apply(create_answer, axis=1)

C:\Users\Admin\AppData\Local\Temp\ipykernel_21016\4158097922.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows_title['answer'] = selected_rows_title.apply(create_answer, axis=1)


In [26]:
selected_rows_title = selected_rows_title.drop(['date_answer', 'field', 'description', 'reference', 'quote', 'conclusion'], axis=1)
selected_rows_title.reset_index(drop=True, inplace=True)
selected_rows_title.sample(5)

,_id,title,source_url,answer
12,6663569ec2f544363eec67d7,Ngồi trên xe máy không đội mũ bảo hiểm bị phạt...,https://thuvienphapluat.vn/hoi-dap-phap-luat/1...,Theo quy định tại điểm i khoản 3 Điều 6 Nghị đ...
10,6663569dc2f544363eec53e2,Có phạt đối với người ngồi trên xe máy không đ...,https://thuvienphapluat.vn/hoi-dap-phap-luat/4...,Theo Khoản 3 Điều 11 Nghị định 100/2019/NĐ-CP ...
19,6663573ac2f544363eeea0b6,Có được nộp phạt tại chỗ khi chạy xe máy không...,https://thuvienphapluat.vn/hoi-dap-phap-luat/4...,"Tại Điểm i, Điểm k Khoản 2 Điều 6 Nghị định 10..."
15,6663569fc2f544363eec876e,Chở người bằng xe máy tham gia giao thông khôn...,https://thuvienphapluat.vn/hoi-dap-phap-luat/8...,Căn cứ theo điểm o khoản 3 Điều 6 Nghị định 10...
1,6663569ac2f544363eec23c3,Người đi xe đạp điện không đội mũ bảo hiểm bị ...,https://thuvienphapluat.vn/hoi-dap-phap-luat/5...,Theo Điều 8 Nghị định 100/2019/NĐ-CP được bổ s...


In [27]:
sentences_answer = []
ids_answer = []

for idx in selected_rows_title.index:
    sentences_answer.append(selected_rows_title.at[idx, 'answer'])
    ids_answer.append(selected_rows_title.at[idx, '_id'])

In [28]:
segmented_sentences_answer = [word_segment(sentence) for sentence in sentences_answer]

In [29]:
encoded_input_answer = tokenizer(segmented_sentences_answer, padding=True, truncation=True, max_length=255,  return_tensors='pt')
encoded_input_answer = {key: value.to('cuda') for key, value in encoded_input_answer.items()}

In [30]:
with torch.no_grad():
    model_output_answer = model(**encoded_input_answer)

In [31]:
embeddings_answer = mean_pooling(model_output_answer, encoded_input_answer['attention_mask'])
embeddings_answer = embeddings_answer.cpu().numpy()

In [32]:
from sklearn.metrics.pairwise import cosine_similarity

similarities_answer = cosine_similarity(embeddings_query, embeddings_answer)

In [33]:
top_20_indices_answer = np.argsort(similarities_answer[0])[::-1][:20]

for idx in top_20_indices_answer:
    similarity_score = similarities_answer[0][idx]
    similar_sentence = sentences_answer[idx]
    doc_id = ids_title[idx]
    print(f"ID: {doc_id}, Similarity Score: {similarity_score:.4f}, Sentence: {similar_sentence}")

ID: 6663569ac2f544363eec2335, Similarity Score: 0.8311, Sentence: Theo Khoản 3 Điều 11 Nghị định 100/2019/NĐ-CP quy định phạt tiền từ 200.000 đồng đến 300.000 đồng đối với người được chở trên xe mô tô, xe gắn máy (kể cả xe máy điện), các loại xe tương tự xe mô tô, các loại xe tương tự xe gắn máy, xe đạp máy (kể cả xe đạp điện) không đội “mũ bảo hiểm cho người đi mô tô, xe máy” hoặc đội “mũ bảo hiểm cho người đi mô tô, xe máy” không cài quai đúng quy cách khi tham gia giao thông trên đường bộ. Như vậy, khi bạn ngồi trên xe máy mà không đội mũ bảo hiểm thì bạn sẽ bị phạt tiền từ 200.000 đồng đến 300.000 đồng. Ngoài ra, xử phạt bạn với hành vi không đội mũ bảo hiểm thì người chở bạn cũng bị phạt tiền từ 200.000 đồng đến 300.000 đồng.
ID: 6663569ac2f544363eec23c2, Similarity Score: 0.8282, Sentence: Theo quy định tại điểm i khoản 3 Điều 6 Nghị định 171/2013/NĐ-CP về quy định xử phạt vi phạm hành chính trong lĩnh vực giao thông đường bộ, đường sắt có hiệu lực từ ngày 1/1/2014 thì điều khiển

# Caculate the final similarity score

In [34]:
score_title_ids = []
for idx in top_20_indices_title:
    score_title_ids.append({
        'id': ids_title[idx],
        'score': similarities_title[0][idx]
    })

In [35]:
score_answer_ids = []
for idx in top_20_indices_answer:
    score_answer_ids.append({
        'id': ids_answer[idx],
        'score': similarities_answer[0][idx]
    })

In [36]:
title_scores_by_id = {score_dict['id']: score_dict['score'] for score_dict in score_title_ids}
answer_scores_by_id = {score_dict['id']: score_dict['score'] for score_dict in score_answer_ids}

score_sbert_ids = []

for id in set(title_scores_by_id.keys()).intersection(answer_scores_by_id.keys()):
    final_score = title_scores_by_id[id] * answer_scores_by_id[id]
    score_sbert_ids.append({'id': id, 'score': final_score})

score_sbert_ids = sorted(score_sbert_ids, key=lambda x: x['score'], reverse=True)

In [37]:
for score_dict in score_sbert_ids[:10]:
    print(f"ID: {score_dict['id']}, Sbert Score: {score_dict['score']:.4f}")

ID: 6663569dc2f544363eec6414, Sbert Score: 0.6124
ID: 6663569dc2f544363eec53e2, Sbert Score: 0.5794
ID: 6663569cc2f544363eec510a, Sbert Score: 0.5736
ID: 6663569fc2f544363eec7b41, Sbert Score: 0.5272
ID: 6663569cc2f544363eec4985, Sbert Score: 0.5213
ID: 6663569bc2f544363eec3bf5, Sbert Score: 0.5134
ID: 6663573ac2f544363eeea0b6, Sbert Score: 0.5093
ID: 6663569ac2f544363eec2486, Sbert Score: 0.4963
ID: 6663569fc2f544363eec876e, Sbert Score: 0.4691
ID: 6663569ac2f544363eec2236, Sbert Score: 0.4593


In [38]:
score_bm25_ids = [{'id': doc_dict['id'], 'score': doc_score} for doc_dict, doc_score in zip(top_docs, doc_scores)]

In [39]:
final_scores = []
for score_sbert in score_sbert_ids:
    for score_bm25 in score_bm25_ids:
        score_sbert_id = str(score_sbert['id'])
        if score_sbert_id == score_bm25['id']:
            final_score = score_sbert['score'] * score_bm25['score']
            final_scores.append({'id': score_sbert_id, 'final_score': final_score})
            break

final_scores = sorted(final_scores, key=lambda x: x['final_score'], reverse=True)

for score_dict in final_scores[:5]:
    print(f"ID: {score_dict['id']}, Final Score: {score_dict['final_score']:.4f}")


ID: 6663569dc2f544363eec6414, Final Score: 211.1196
ID: 6663569cc2f544363eec510a, Final Score: 199.8013
ID: 6663569dc2f544363eec53e2, Final Score: 199.7391
ID: 6663569cc2f544363eec4985, Final Score: 180.9425
ID: 6663569fc2f544363eec7b41, Final Score: 178.3325
